# REQUIRED LIBRARIES

In [1]:
import cv2
import numpy as np

# INTRODUCTION TO CV2 VIDEO PART

In [2]:
i=0
video=cv2.VideoCapture('intrusion video.mp4')
while(1):
    ret,frame=video.read()
    if ret==0:
        break
    i=i+1
print("number of frames is ",i)

number of frames is  263


In [3]:
video=cv2.VideoCapture('intrusion video.mp4')
rate=video.get(cv2.CAP_PROP_FPS)
count=video.get(cv2.CAP_PROP_FRAME_COUNT)
duration=count/rate
duration

8.801366666666667

In [4]:
count,rate

(263.0, 29.881722914244378)

In [5]:
video=cv2.VideoCapture('intrusion video.mp4')
width,height=video.get(cv2.CAP_PROP_FRAME_WIDTH),video.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [6]:
width,height

(1280.0, 720.0)

In [7]:
video=cv2.VideoCapture('intrusion video.mp4')
ret,frame=video.read()
frame.shape

(720, 1280, 3)

# BACKGROUND SUBTRACTION (SINGLE FRAME REFERENCE)

In [8]:
def background(frame):
    mean=frame
    var=9*np.ones(mean.shape,dtype=int)
    return mean,var

In [9]:
def foregroundDetection(new_frame,mean,var,lamb):
    d=mean-new_frame
    y=var*lamb**2
    I=np.square(d)-y
    mask=np.all(I>0,axis=2)
    rI=255*mask.astype(int)
    rI=rI.astype(np.uint8)
    return rI

In [10]:
def background_Subtraction(mean,var,lamb):
    out=cv2.VideoWriter('detected Video.avi',cv2.VideoWriter_fourcc(*'XVID'),rate,(1280, 720))
    #now detecting
    video=cv2.VideoCapture('intrusion video.mp4')
    new_arr=np.zeros((720,1280,3),dtype=np.uint8)
    while(1):
        ret,frame=video.read()
        if ret==0:
            break
        rI=foregroundDetection(frame,mean,var,lamb)
        cI=cv2.fastNlMeansDenoising(rI)
        new_arr[:,:,0]=cI
        new_arr[:,:,1]=cI
        new_arr[:,:,2]=cI
        out.write(new_arr)
    out.release()

In [11]:
video=cv2.VideoCapture('intrusion video.mp4')
#first frame is considered as background image
ret,ref_frame=video.read()
mean,var=background(ref_frame)
background_Subtraction(mean,var,3)

# BACKGROUND SUBSTRACTION (UNIMODAL MODELLING)

In [12]:
def meanVarUpdate(old_mean,old_var,cI,frame,alpha):
    new_mean=np.zeros(frame.shape)
    new_var=np.zeros(frame.shape)
    d=np.square(frame-old_mean)
   # idx=np.where(cI==0)
    new_mean=old_mean*(1-alpha)+frame*alpha
    new_var=(1-alpha)*(old_var+d*alpha)
    new_var=np.clip(new_var,a_min=9,a_max=None)
    return new_mean,new_var

In [13]:
def background_Subtraction_uni(mean,var,lamb,alpha):
    out=cv2.VideoWriter('detected Video Uni.avi',cv2.VideoWriter_fourcc(*'XVID'),rate,(1280, 720))
    #now detecting
    video=cv2.VideoCapture('intrusion video.mp4')
    lamb=3
    new_arr=np.zeros((720,1280,3),dtype=np.uint8)
    ret,frame=video.read()
    while(1):
        ret,frame=video.read()
        if ret==0:
            break
        rI=foregroundDetection(frame,mean,var,lamb)
        cI=cv2.fastNlMeansDenoising(rI,None, 10, 10, 7)
        cI=cI.astype(np.uint8)
        new_arr[:,:,0]=cI
        new_arr[:,:,1]=cI
        new_arr[:,:,2]=cI
        mean,var=meanVarUpdate(mean,var,cI,frame,alpha)
        out.write(new_arr)
    out.release()

In [14]:
video=cv2.VideoCapture('intrusion video.mp4')
#first frame is considered as background image
ret,frame=video.read()
ret,new_frame=video.read()
mean,var=background(frame)
background_Subtraction_uni(mean,var,0.8,0.8)